In [3]:
import requests
import datetime
import time
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

Initial steps if running on Google Colab, to download support files from GitHub and set the working directory

In [2]:
#for running on Colab
#!git clone https://github.com/MBWestcott/gas-forecast.git

# 2. Change into the repo directory
%cd /content/gas-forecast/notebooks



[WinError 3] The system cannot find the path specified: '/content/gas-forecast/notebooks'
d:\dev\gas-forecast\notebooks


### First download the raw data from the National Gas data portal

In [5]:

raw_data_folder = Path("../data/raw/")

def download_csv(url, output_file):
    """
    Downloads a CSV file from the given URL and saves it to the specified file.

    :param url: URL to download the CSV data from.
    :param output_file: Path to the local file where the CSV will be saved.
    """
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Ensure we notice bad responses

        # Write the content (CSV data) to a file in binary mode
        with open(output_file, 'wb') as f:
            f.write(response.content)

        print(f"CSV file has been successfully downloaded and saved as '{output_file}'.")

    except requests.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"An error occurred: {err}")


def download_raw_data():
    pubIdsFile = Path("../PUB ids.txt")
    with open(pubIdsFile) as f:
        pubIds = f.read()
        pubIds = pubIds.replace("\n", ",").strip()

    earliest = datetime.date(2020,4,1) # Download data going back 5 years
    
    download_from = datetime.date.today().replace(day=1) # start first download on first day of current month
    download_to = datetime.date.today() # end first download on today's date
    while(download_from > earliest):

        # Format the date in yyyy-mm-dd format
        formatted_from = download_from.strftime("%Y-%m-%d")
        formatted_to = download_to.strftime("%Y-%m-%d")

        csv_url = f"https://data.nationalgas.com/api/find-gas-data-download?applicableFor=Y&dateFrom={formatted_from}&dateTo={formatted_to}&dateType=GASDAY&latestFlag=Y&ids={pubIds}&type=CSV"
        month_format = download_from.strftime("%Y-%m")
        output_filename = raw_data_folder /  f"{month_format}.csv"

        download_csv(csv_url, output_filename)
        time.sleep(2) # brief courtesy sleep
        download_to = download_from - datetime.timedelta(days=1) # next download should go up to the day before the previous download start date
        download_from = download_to.replace(day=1) # next download should start on the first day of the month

# Do the download if the raw data is not there already
csvCount = sum(1 for f in raw_data_folder.iterdir() if f.is_file() and f.suffix == '.csv')
if(csvCount < 60):
    download_raw_data()

### Load the raw data

Load the raw CSVs into a single and dataframe, pivot it so that each column represents a feature.
Rename the Applicable At date field to Gas Day, and rename the columns that are going to be reused for ground truth and time series

In [6]:
label_cols = ["SAP", "SMPBuy", "SMPSell"]

def pivot(df, cols):

    #only keep the values we are interested in
    mask = df["Data Item"].isin(cols)

    df_filtered = df[mask]

    # if there are duplicates for the field and gas day, take the latest
    df_latest = (
        df_filtered
        .sort_values("Applicable At")
        .groupby(["Gas Day", "Data Item"])
        .last()  # this takes the row with the highest (i.e. latest) "Applicable At" per group
        .reset_index()
    )

    # pivot to get 1 row per gas day
    df_latest = df_latest.pivot(index="Gas Day", columns="Data Item", values="Value").reset_index()

    # Drop 1 column that accounts for most of the NaNs
    #df_latest.drop(columns=["Composite Weather Variable - Actual"], inplace=True)

    return df_latest

def load_data():
    #Read raw CSVs
    pathlist = list(Path(raw_data_folder).rglob('*.csv'))
    file_count = len(pathlist)
    dfs = []
    files_done = 0
    for path_obj in pathlist:
        path = str(path_obj)

        df = pd.read_csv(path,
            parse_dates=["Applicable At", "Applicable For", "Generated Time"],
            dayfirst=True)

        df.rename(columns={'Applicable For': 'Gas Day'}, inplace=True)
        df['Gas Day'] = pd.to_datetime(df['Gas Day'], dayfirst=True)
        # daily summary columns:

        daily_cols = df["Data Item"].unique()
        # print(daily_cols)
        # Get price and demand columns, to use as tommorow's ground truth, and with 1-3 days lag

        #label_cols = ["SAP, Actual Day", "SMP Buy, Actual Day", "SMP Sell, Actual Day", "Demand Actual, NTS, D+1"]

        #df_labels = pivot(df, label_cols)


        df_daily = pivot(df, daily_cols)
        dfs.append(df_daily)

        files_done += 1
        if files_done % 10 == 0:
            print(f"Processed {files_done} of {file_count} raw files")

    df = pd.concat(dfs)

    #Rename the columns that are going to be reused for ground truth and time series
    df.rename(columns={"SAP, Actual Day": 'SAP', "SMP Buy, Actual Day": 'SMPBuy', "SMP Sell, Actual Day": 'SMPSell'}, inplace=True)
    return df

df = load_data()
df.to_csv(Path("../data/processed/pivoted.csv"), index=False)
df.info()

Processed 10 of 60 raw files
Processed 20 of 60 raw files
Processed 30 of 60 raw files
Processed 40 of 60 raw files
Processed 50 of 60 raw files
Processed 60 of 60 raw files


PermissionError: [Errno 13] Permission denied: '..\\data\\processed\\pivoted.csv'

### Preprocess data

Add the previous 5 days' prices as lag features, and 7- and 30-day rolling averages and standard deviations. Also add day of week features, and a cyclical coding of the day of year for seasonality.

In [7]:
def preprocess(df, add_lags=True, add_labels=True):

    """Deal with missing values, add lagged features, rolling averages and stds, Day of Week, and cyclic encoding for seasonality"""

    if add_lags:
        lag_days = 5
        for i in range(1, lag_days+1):
            for col in label_cols:
                df[f"{col} D-{i}"] = df[col].shift(i)

        # add rolling averages and stds
        for col in label_cols:
            for window in [7, 30]:
                df[f'{col} D{window} roll mean'] = (
                    df[col]
                    .shift(1)               # so today's feature doesn't include today's price
                    .rolling(window=window, min_periods=1)  
                    .mean()
                    )
                df[f'{col} D{window} roll std'] = (
                    df[col]
                    .shift(1)               # so today's feature doesn't include today's price
                    .rolling(window=window, min_periods=1)  
                    .std()
                )

    # add day of week
    df['Day of Week'] = df['Gas Day'].dt.weekday
    df['Is Weekday'] = (df['Gas Day'].dt.weekday < 5).astype(int)
    df['Next Day Is Weekday'] = ((df['Gas Day'] + pd.Timedelta(days=1)).dt.weekday < 5).astype(int)
    # cyclic encoding for seasonality
    df['Day of Year'] = df['Gas Day'].dt.dayofyear
    df['sin_DoY'] = np.sin(2 * np.pi * df['Day of Year'] / 365)
    df['cos_DoY'] = np.cos(2 * np.pi * df['Day of Year'] / 365)

    if add_labels:
        # Add labels for next day's actuals
        for col in label_cols:
            df[f"Next Day {col}"] = df[col].shift(-1)

    return df

df = preprocess(df)
df.to_csv(Path("../data/processed/preprocessed.csv"), index=False)
df.head()

PermissionError: [Errno 13] Permission denied: '..\\data\\processed\\preprocessed.csv'

### Clean missing values and outliers

Most of the missing values are missing "Composite Weather Variable - Actual" from 2020-21. These affect around 15% of the dataset. Best way to fill in those is with the Normal forecast, which should usually be the closest. Apart from that there are very few missing readings so it is feasible to discard any remaining rows with missing data (done at the end, to avoid introducing errors into the lag features)

Also remove outliers where any of the prices was 0, and one of the next day prices was more 50% away from the current day's price

In [8]:
def clean(df, remove_outliers=True):
    # fill missing CWV actuals with the normal forecast
    df['Composite Weather Variable - Actual'] = df['Composite Weather Variable - Actual'].fillna(df['Composite Weather Variable - Normal'])

    # There should be very remaining few rows that have any NaNs so we can drop any that do
    df.dropna(inplace=True)

    # Can drop the composite weather forecasts
    df.drop(columns=["Composite Weather Variable - Normal", "Composite Weather Variable - Cold", "Composite Weather Variable - Warm"], inplace=True)

    if(remove_outliers):
        for col in label_cols:    
            # remove outliers where any of the prices was 0
            print(df.shape)
            df = df[df[col] != 0]
            print(df.shape)
            df = df[df[f"Next Day {col}"] != 0]
            print(df.shape)
            #... and where the next day price is more than least 50% away from the current day's price
            df = df[abs(df[col] - df[f"Next Day {col}"])/df[col] < 0.5]
            print(df.shape)
    return df    

df = clean(df)
df.to_csv(Path("../data/processed/preprocessed_and_cleaned.csv"), index=False)
df.head()

(1802, 78)
(1802, 78)
(1802, 78)
(1789, 78)
(1789, 78)
(1789, 78)
(1789, 78)
(1784, 78)
(1784, 78)
(1783, 78)
(1783, 78)
(1762, 78)


Data Item,Gas Day,Aggregate LNG Importations - Daily Flow,Beach Including Norway - Daily Flow,Beach and IOG - Beach Delivery,Beach and IOG - Daily Flow,Composite Weather Variable - Actual,Demand - Cold,"Demand - Cold, (excluding interconnector and storage)",Demand - Warm,"Demand - Warm, (excluding interconnector and storage)",...,SMPSell D30 roll std,Day of Week,Is Weekday,Next Day Is Weekday,Day of Year,sin_DoY,cos_DoY,Next Day SAP,Next Day SMPBuy,Next Day SMPSell
5,2020-05-06,50.23730,142.20118,192.43848,192.43848,12.0645,245.399580,216.150490,145.432835,116.183744,...,0.005142,2,1,1,127,0.816538,-0.577292,0.4834,0.5187,0.4481
6,2020-05-07,53.59770,141.87295,195.47065,195.47065,13.4655,243.927941,214.341578,145.153439,115.567076,...,0.011180,3,1,1,128,0.806480,-0.591261,0.4756,0.5109,0.4403
7,2020-05-08,51.08822,135.19872,186.28694,186.28694,15.5400,171.000000,131.000000,154.000000,114.000000,...,0.010249,4,1,0,129,0.796183,-0.605056,0.4722,0.5075,0.4369
8,2020-05-09,53.28634,127.04213,180.32847,180.32847,15.0800,172.000000,134.000000,141.000000,104.000000,...,0.009697,5,0,0,130,0.785650,-0.618671,0.4615,0.4968,0.4262
9,2020-05-10,53.14522,127.29315,180.43837,180.43837,12.6200,264.115666,226.223933,180.336933,142.445199,...,0.009489,6,0,1,131,0.774884,-0.632103,0.4569,0.4922,0.4216


### Split the data into training and test sets
Various ways were investigated:
- Use the data before a cutoff date to train, and after that date to test. Designed to test whether the model will generalise to the most recent period, despite having been trained on earlier periods
- Split the data 50/50 by date. Use all of the earlier half, and a random half of the later half, to train. Use the remaining half of the later half to test. Designed as a compromise between the above approach and a random split
- Split the data randomly regardless of date

In [29]:

def split_train_test_on_date(df, split_date, discard_before_date):
    """
    Splits the DataFrame into training set (gas days before split date) and test set (ga days fron the split date on)

    :param df: The DataFrame to split.
    :param split_date: The date to split the DataFrame on.
    :param discard_before_date: Discard anything before this date. Added to exclude time of Covid lockdowns.
    :return: Tuple of (training set, testing set).
    """

    # Split the DataFrame into training and testing sets
    train_df = df[df['Gas Day'].between(discard_before_date, split_date, inclusive = "neither")]
    test_df = df[df['Gas Day'] >= split_date]

    return train_df, test_df

def split_with_test_half_of_last_half(df, discard_before_date):
    """
    Use all the earlier half, and half the later half, to train
    Use the other half of the later half to test
    """
    df2 = df[df['Gas Day'] >= discard_before_date]
    mid_date = df2['Gas Day'].mean()
    first_half = df2[df2['Gas Day'] < mid_date]
    second_half = df2[df2['Gas Day'] >= mid_date]
    
    train_df, test_df = train_test_split(second_half, test_size=0.5, shuffle=True)
    train_df = pd.concat([first_half, train_df])
    return train_df, test_df

def n_train_n_test(df, n_train, n_test, discard_before_date):
    """Split based on number or fraction of rows"""
    #df.to_csv("..\\data\\processed\\all.csv", index=False)
    df = df[df['Gas Day'] >= discard_before_date]
    # Split the DataFrame into training and testing sets
    train_df, test_df = train_test_split(df, test_size=n_test, train_size=n_train, shuffle=True)
    #train_df.to_csv("..\\data\\processed\\train.csv", index=False)
    #test_df.to_csv("..\\data\\processed\\test.csv", index=False)
    return train_df, test_df

def get_X(df):
    ys = ["Next Day " + col for col in label_cols]
    df2 = df.drop(columns=ys)
    df2.drop(columns=["Gas Day"], inplace=True)
    # experimentally - just include the price time series columns
    #for col in df2.columns.tolist():      # iterate over a copy of the column list
    #    if not is_price_column(col):             # if the substring isn’t found
    #        df2.drop(columns=col, inplace=True)
    #df2 = df2[label_cols]
    #df2 = df2[["SAP"]]
    return df2

#def is_price_column(column_name):
#    if "SMP" in column_name or "SAP" in column_name:
#        return True
#    return False



# split on date, or random proportions
#train, test = split_train_test(df, '2024-10-01', '2021-03-01')
#train, test = n_train_n_test(df, n_train=250, n_test=50, discard_before_date='2021-03-01')
#train, test = n_train_n_test(df, n_train=0.75, n_test=0.25, discard_before_date='2021-03-01')
#train = train[train['Gas Day'] > '2021-03-01']
#test = test[test['Gas Day'] > '2021-03-01']
train, test = split_with_test_half_of_last_half(df, '2023-09-01')
X_train = get_X(train)
X_test = get_X(test)


### Use Root Mean Squared Error as the measure of accuracy

This is appropriate to price forecasting because it penalises larger inaccuracies

In [30]:
# Root mean squared error - penalises larger errors more than smaller ones
def get_rmse(actuals, predictions):
    rmse =  np.sqrt(np.mean((predictions - actuals)**2))
    return round(rmse, 4)

#Mean absolute percentage error
def get_mape(actuals, predictions):
    mape = np.mean(np.abs((predictions - actuals) / actuals)) * 100
    return round(mape, 4)


def print_model_stats(model, X):

    # 1. Coefficients and intercept
    if hasattr(model, "coef_"):
        #print("Coefficients:", model.coef_)      # array of shape (n_features,)
        cdf = pd.DataFrame(model.coef_, X.columns, columns=['Coefficients'])
        cdf = cdf.sort_values(by='Coefficients', ascending=False)
        print(cdf)
    if hasattr(model, "intercept_"):
        print("Intercept:", model.intercept_)    # scalar (or array if multi-output)

    # 2. Model parameters
    print("Parameters:", model.get_params())

    # 3. Data‐related attributes
    #print("Number of features seen during fit:", model.n_features_in_)
    #if hasattr(model, "feature_names_in_"):
    #    print("Feature names:", model.feature_names_in_)

    # 4. Linear algebra internals (rarely needed)
    if hasattr(model, "rank_"):
        print("Rank of design matrix:", model.rank_)
    if hasattr(model, "singular_"):
        print("Singular values of X:", model.singular_)

### Set up a framework to train models, and compare their performance on the test dataset against a naive predictor

The naive predictor takes the current day's System Average Price and System Marginal Prices as the predictions for the next day

In [39]:
# Use the previous day's actual as a naive predictor

def get_y(df, col):
    return df["Next Day " + col]

def test_model(model, X, y):
    y_pred = model.predict(X)
    rmse = get_rmse(y, y_pred)
    return rmse

def train_and_test_model(model, df_train, df_test, col):
    X_train = get_X(df_train)
    X_test = get_X(df_test)
    y_train = get_y(df_train, col)
    y_test = get_y(df_test, col)
    #scaler = StandardScaler()
    #X_train_scaled = scaler.fit_transform(X_train)
    #X_test_scaled = scaler.fit_transform(X_test)
    X_train_scaled = X_train
    X_test_scaled = X_test

    model.fit(X_train_scaled, y_train)

    rmse_train = test_model(model, X_train_scaled, y_train)
    rmse_test = test_model(model, X_test_scaled, y_test)

    return model, rmse_train, rmse_test

def train_test_and_report_for_prices(model_factory, df_train, df_test, include_model_stats=True):
    for col in label_cols:
        # Instantiate model.
        model = model_factory()

        # Train and test it
        model, rmse_train, rmse_test = train_and_test_model(model, df_train, df_test, col)

        # Print model details
        if include_model_stats:
            X_train = get_X(df_train)
            print_model_stats(model, X_train)

        # Get naive prediction stats for comparison
        rmse_naive_train, rmse_naive_test = naive_predictions(df_train, df_test, col)

        print_results(col + " train", rmse_naive_train, rmse_train)
        print_results(col + " test", rmse_naive_test, rmse_test)

def naive_predictions(df_train, df_test, col):
    naive_predictions_train = df_train[col]
    actuals_train = df_train[f"Next Day {col}"]
    #mape_naive_train = get_mape(actuals_train, naive_predictions_train)
    #print(f"MAPE train (naive predictor) for {col}: {mape_naive_train}")
    rmse_naive_train = get_rmse(actuals_train, naive_predictions_train)
    #print(f"RMSE train (naive predictor) for {col}: {rmse_naive_train}")

    naive_predictions_test = df_test[col]
    actuals_test = df_test[f"Next Day {col}"]
    #mape_naive_test = get_mape(actuals_test, naive_predictions_test)
    #print(f"MAPE test (naive predictor) for {col}: {mape_naive_test}")
    rmse_naive_test = get_rmse(actuals_test, naive_predictions_test)
    #print(f"RMSE test (naive predictor) for {col}: {rmse_naive_test}")
    return rmse_naive_train, rmse_naive_test

def print_results(case, rmse_naive, rmse_model):
    headline = "Worse" if rmse_naive <= rmse_model else "Better"
    print(f"{case} - {headline} - model {rmse_model} v naive {rmse_naive}")


### Try linear regression models

...to predict each of SAP (System Average Price), SMPBuy (System Marginal Price - Buy) and SMPSell (System Marginal Price - Sell). This generally performs worse than the naive predictor in testing, especially using a date-based split

In [49]:
print ("Linear regression model:")
model_factory = lambda: LinearRegression()

print("Using random train-test split...")
train, test = n_train_n_test(df, n_train=0.75, n_test=0.25, discard_before_date='2021-03-01')
train_test_and_report_for_prices(model_factory, train, test, include_model_stats=False)

print("Using pure date-based train-test split...")
train, test = split_train_test_on_date(df, '2024-04-01', '2020-10-01')
train_test_and_report_for_prices(model_factory, train, test, include_model_stats=False)

print("Using half of the last half for testing...")
train, test = split_with_test_half_of_last_half(df, '2021-04-01')
train_test_and_report_for_prices(model_factory, train, test, include_model_stats=False)


Linear regression model:
Using random train-test split...
SAP train - Better - model 0.3856 v naive 0.4819
SAP test - Worse - model 0.3817 v naive 0.3699
SMPBuy train - Better - model 0.4434 v naive 0.534
SMPBuy test - Worse - model 0.521 v naive 0.4728
SMPSell train - Better - model 0.5614 v naive 0.6358
SMPSell test - Worse - model 0.5344 v naive 0.4612
Using pure date-based train-test split...
SAP train - Better - model 0.4075 v naive 0.4965
SAP test - Worse - model 0.6024 v naive 0.0806
SMPBuy train - Better - model 0.4878 v naive 0.5645
SMPBuy test - Worse - model 0.8553 v naive 0.0966
SMPSell train - Better - model 0.5906 v naive 0.6496
SMPSell test - Worse - model 1.0567 v naive 0.0891
Using half of the last half for testing...
SAP train - Better - model 0.434 v naive 0.529
SAP test - Worse - model 0.1588 v naive 0.1274
SMPBuy train - Better - model 0.5215 v naive 0.6017
SMPBuy test - Worse - model 0.1892 v naive 0.1475
SMPSell train - Better - model 0.6307 v naive 0.6935
SMPSel

### Try a random forest model
Linear regression generally performed worse than the naive predictor in testing, especially using a date-based split, so let's try a random forest model. The hyperparameters for the best version were obtained by random search in the second code block below

In [ ]:
from sklearn.ensemble import RandomForestRegressor
print ("Random forest model:")
#RandomForestRegressor(n_estimators = 500, min_samples_split = 2, min_samples_leaf= 2, max_features = 0.9, max_depth = 20, ccp_alpha = 0.0) # best from random searh
#RandomForestRegressor(n_estimators = 200, min_samples_split = 2, min_samples_leaf= 2, max_features = 0.7, max_depth = 20, ccp_alpha = 0.0) # best for SAP: SAP test - Better - model 0.77 v naive 0.8
model_factory = lambda: RandomForestRegressor(n_estimators = 500, min_samples_split = 2, min_samples_leaf= 2, max_features = 0.9, max_depth = 20, ccp_alpha = 0.0)

print("Using random train-test split...")
train, test = n_train_n_test(df, n_train=0.75, n_test=0.25, discard_before_date='2021-03-01')
train_test_and_report_for_prices(model_factory, train, test, include_model_stats=False)

print("Using pure date-based train-test split...")
train, test = split_train_test_on_date(df, '2024-04-01', '2020-10-01')
train_test_and_report_for_prices(model_factory, train, test, include_model_stats=False)

print("Using half of the last half for testing...")
train, test = split_with_test_half_of_last_half(df, '2021-04-01')
train_test_and_report_for_prices(model_factory, train, test, include_model_stats=False)

Random forest model:
Using random train-test split...
SAP train - Better - model 0.1844 v naive 0.4545
SAP test - Better - model 0.4361 v naive 0.4623
SMPBuy train - Better - model 0.2174 v naive 0.5183
SMPBuy test - Better - model 0.4992 v naive 0.5225
SMPSell train - Better - model 0.2703 v naive 0.5929
SMPSell test - Worse - model 0.6279 v naive 0.6088
Using pure date-based train-test split...
SAP train - Better - model 0.1958 v naive 0.4965
SAP test - Worse - model 0.1021 v naive 0.0806
SMPBuy train - Better - model 0.2334 v naive 0.5645
SMPBuy test - Worse - model 0.1236 v naive 0.0966
SMPSell train - Better - model 0.2922 v naive 0.6496
SMPSell test - Worse - model 0.1203 v naive 0.0891
Using half of the last half for testing...
SAP train - Better - model 0.2096 v naive 0.5302
SAP test - Worse - model 0.1186 v naive 0.1116
SMPBuy train - Better - model 0.2452 v naive 0.6026
SMPBuy test - Worse - model 0.1443 v naive 0.1366
SMPSell train - Better - model 0.3105 v naive 0.6943
SMPS

In [ ]:
#Random search for hyperparameter tuning
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error

param_grid = {
    'n_estimators':     [100, 200, 500],
    'max_depth':        [None, 10, 20],
    'min_samples_split':[2],
    'min_samples_leaf': [2],
    'max_features':     [0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 1.0],
    'ccp_alpha':        [0.0, 0.001]
}
#param_grid = {
#    'n_estimators':     [100, 200],
#    'max_depth':        [None, 10, 20],
#    'min_samples_split':[2, 5],
#    'min_samples_leaf': [1, 2],
#    'max_features':     ['sqrt'],
#    'ccp_alpha':        [0.001, 0.01]
#}
X_train = get_X(train)
for col in label_cols:

    rf = RandomForestRegressor(
        random_state=42,
        n_jobs=-1,
        oob_score=True   # optional: get out‑of‑bag score on your train set
    )

    #grid = GridSearchCV(
    #    estimator=rf,
    #    param_grid=param_grid,
    #    cv=5,                             # 5‑fold CV on X_train/y_train
    #    scoring='neg_root_mean_squared_error',
    #    n_jobs=-1,
    #    verbose=3
    #)
    rand_search = RandomizedSearchCV(
        estimator=rf,
        param_distributions=param_grid,             
        n_iter=50,                                  
        cv=5,
        scoring='neg_root_mean_squared_error',      
        n_jobs=-1,
        verbose=3,
        random_state=42
    )

    y_train = get_y(train, col)
    rand_search.fit(X_train, y_train)

    print("Best hyperparameters:", rand_search.best_params_)
    print("Best CV RMSE on train set: {:.4f}".format(-rand_search.best_score_))


    X_test = get_X(test)
    y_test = get_y(test, col)
    # -----------------------------------------------------------------------------
    # 5. Evaluate the best model on the TEST set
    # -----------------------------------------------------------------------------
    best_model = rand_search.best_estimator_

    rmse_test = test_model(best_model, X_test, y_test)
    rmse_naive_train, rmse_naive_test = naive_predictions(train, test, col)

    print_results(col + " test", rmse_naive_test, rmse_test)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best hyperparameters: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 0.7, 'max_depth': 10, 'ccp_alpha': 0.0}
Best CV RMSE on train set: 0.4529
SAP test - Worse - model 0.3753 v naive 0.3689
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best hyperparameters: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 1.0, 'max_depth': None, 'ccp_alpha': 0.0}
Best CV RMSE on train set: 0.5374
SMPBuy test - Worse - model 0.4247 v naive 0.4164
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best hyperparameters: {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 0.5, 'max_depth': 20, 'ccp_alpha': 0.0}
Best CV RMSE on train set: 0.6426
SMPSell test - Worse - model 0.4858 v naive 0.4636


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best hyperparameters: {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 1.0, 'max_depth': 20, 'ccp_alpha': 0.0}
Best CV RMSE on train set: 0.7879
SAP test - Worse - model 0.78 v naive 0.78
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best hyperparameters: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 0.7, 'max_depth': None, 'ccp_alpha': 0.001}
Best CV RMSE on train set: 0.8540
SMPBuy test - Worse - model 0.86 v naive 0.83
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best hyperparameters: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 0.5, 'max_depth': None, 'ccp_alpha': 0.001}
Best CV RMSE on train set: 0.8906
SMPSell test - Worse - model 0.9 v naive 0.83

### Next, try gradient booster


In [51]:
from xgboost import XGBRegressor

print("Gradient Boosting model (XGBoost XGBRegressor):")

model_factory = lambda: XGBRegressor(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.7,
        reg_alpha=0.0,
        reg_lambda=1.0,
        random_state=42
    )

print("Using random train-test split...")
train, test = n_train_n_test(df, n_train=0.75, n_test=0.25, discard_before_date='2021-03-01')
train_test_and_report_for_prices(model_factory, train, test, include_model_stats=False)

print("Using pure date-based train-test split...")
train, test = split_train_test_on_date(df, '2024-04-01', '2020-10-01')
train_test_and_report_for_prices(model_factory, train, test, include_model_stats=False)

print("Using half of the last half for testing...")
train, test = split_with_test_half_of_last_half(df, '2021-04-01')
train_test_and_report_for_prices(model_factory, train, test, include_model_stats=False)


Gradient Boosting model (XGBoost XGBRegressor):
Using random train-test split...
SAP train - Better - model 0.0242 v naive 0.4618
SAP test - Worse - model 0.453 v naive 0.44
SMPBuy train - Better - model 0.0277 v naive 0.5259
SMPBuy test - Worse - model 0.5249 v naive 0.4991
SMPSell train - Better - model 0.0298 v naive 0.6108
SMPSell test - Worse - model 0.589 v naive 0.5532
Using pure date-based train-test split...
SAP train - Better - model 0.0332 v naive 0.4965
SAP test - Worse - model 0.116 v naive 0.0806
SMPBuy train - Better - model 0.0373 v naive 0.5645
SMPBuy test - Worse - model 0.1506 v naive 0.0966
SMPSell train - Better - model 0.0401 v naive 0.6496
SMPSell test - Worse - model 0.1448 v naive 0.0891
Using half of the last half for testing...
SAP train - Better - model 0.028 v naive 0.5301
SAP test - Worse - model 0.1255 v naive 0.1134
SMPBuy train - Better - model 0.033 v naive 0.6027
SMPBuy test - Worse - model 0.1502 v naive 0.1348
SMPSell train - Better - model 0.0322 v

In [ ]:
from sklearn.metrics import mean_squared_error
def naive_for_tcn(y_seq, y_train, y_test, SEQ_LEN):
    # total number of sequences
    n_seq = len(y_seq)
    # number of train sequences
    n_train = len(y_train)

    # map each sequence to its target’s original df index:
    #   sequences start at df index [0..], target of seq i is df index i+SEQ_LEN
    target_idx = np.arange(SEQ_LEN, SEQ_LEN + n_seq)

    # split those indices into train / test
    train_target_idx = target_idx[:n_train]
    test_target_idx  = target_idx[n_train:]

    # naive prediction uses “today’s SAP” = df['SAP'] at (target_idx-1)
    naive_train = df['SAP'].iloc[train_target_idx - 1].values
    naive_test  = df['SAP'].iloc[test_target_idx  - 1].values

    # calculate RMSE
    rmse_naive_train = np.sqrt(mean_squared_error(y_train, naive_train))
    rmse_naive_test  = np.sqrt(mean_squared_error(y_test,  naive_test))
    return rmse_naive_train, rmse_naive_test

In [66]:
# 1. Install the TCN layer
#!pip install keras-tcn

import numpy as np
import pandas as pd
#import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.metrics import RootMeanSquaredError
from tcn import TCN
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# For TCN, let it build up its own sequences.
# Reload the data, don't preprocess, and clean without removing outliers
df = load_data()
df = clean(df, remove_outliers=False)

# 2. Suppose you already have a DataFrame `df` with:
#    - 'Gas Day' datetime index or column
#    - features (lags, rolling stats, calendar encodings, …)
#    - target column 'price'

# If 'Gas Day' is a column:
# df['Gas Day'] = pd.to_datetime(df['Gas Day'])
# df.set_index('Gas Day', inplace=True)

# 3. Prepare sequences for TCN
def make_sequences(X, y, seq_len):
    Xs, ys = [], []
    for i in range(len(X) - seq_len):
        # positional slice for X
        Xs.append(X.iloc[i : i + seq_len].values)
        # positional lookup for y
        ys.append(y.iloc[i + seq_len])
    return np.stack(Xs), np.array(ys)
# parameters
SEQ_LEN    = 30          # e.g. use past 30 days to predict next-day price
TEST_SIZE  = 0.3

# 

X = get_X(df)
y = get_y(df, "SAP")


# scale features
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)

# build sequences
X_seq, y_seq = make_sequences(X_scaled, y, SEQ_LEN)

# train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_seq, y_seq, test_size=TEST_SIZE, shuffle=False
)


#naive_predictions_test = df_test[col]
rmse_naive_train, rmse_naive_test = naive_for_tcn(y_seq, y_train, y_test, SEQ_LEN)

# 4. Build a simple TCN model
n_features = X_train.shape[2]

inputs = Input(shape=(SEQ_LEN, n_features))
# TCN defaults: 64 filters, kernel_size=3, 8 stacks with exponentially increasing dilation
tcn_layer = TCN(return_sequences=False)(inputs)
output    = Dense(1)(tcn_layer)

model = Model(inputs, output)
model.compile(optimizer='adam', loss='mse', metrics=[RootMeanSquaredError()])
model.summary()

# 5. Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20, #50
    batch_size=16,
    callbacks=[
        # optional: early stopping
        # tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    ]
)

# 6. Evaluate & predict
loss, rmse = model.evaluate(X_test, y_test)
#y_pred = model.predict(X_test)
print(f"Test RMSE: {rmse:.4f}")

#print_results(col + " train", rmse_naive_train, rmse_train)
print_results("SAP" + " test", rmse_naive_test, rmse_test)

Processed 10 of 60 raw files
Processed 20 of 60 raw files
Processed 30 of 60 raw files
Processed 40 of 60 raw files
Processed 50 of 60 raw files
Processed 60 of 60 raw files


KeyError: "['Next Day SAP', 'Next Day SMPBuy', 'Next Day SMPSell'] not found in axis"

In [ ]:
df = load_data()
df = preprocess(df, add_lags=False, add_labels=False)
df = clean(df, remove_outliers=False)
df = df.sort_values('Gas Day').reset_index(drop=True)
df.head()

Processed 10 of 60 raw files
Processed 20 of 60 raw files
Processed 30 of 60 raw files
Processed 40 of 60 raw files
Processed 50 of 60 raw files
Processed 60 of 60 raw files


Data Item,Gas Day,Aggregate LNG Importations - Daily Flow,Beach Including Norway - Daily Flow,Beach and IOG - Beach Delivery,Beach and IOG - Daily Flow,Composite Weather Variable - Actual,Demand - Cold,"Demand - Cold, (excluding interconnector and storage)",Demand - Warm,"Demand - Warm, (excluding interconnector and storage)",...,"Storage, Short Range, Maximum potential flow","Storage, Short Range, Stock Levels","System Entry Flows, National, Forecast","System Entry Flows, National, Physical",Day of Week,Is Weekday,Next Day Is Weekday,Day of Year,sin_DoY,cos_DoY
0,2020-05-01,66.15110,135.26344,201.41454,201.41454,10.5824,268.090483,240.620483,160.371026,132.901026,...,0.0,0.0,247.492971,253.383732,4,1,0,122,0.863142,-0.504961
1,2020-05-02,58.78630,131.66283,190.44913,190.44913,11.3089,244.938804,217.103350,145.468324,117.632869,...,0.0,0.0,208.688512,229.860588,5,0,0,123,0.854322,-0.519744
2,2020-05-03,56.55015,141.57363,198.12378,198.12378,11.6531,242.854588,214.663679,144.375923,116.185014,...,0.0,0.0,221.676044,252.336835,6,0,1,124,0.845249,-0.534373
3,2020-05-04,52.82721,152.87412,205.70133,205.70133,11.9252,242.098717,213.561444,144.746098,116.208825,...,0.0,0.0,206.768342,231.939225,0,1,1,125,0.835925,-0.548843
4,2020-05-05,62.21188,134.50813,196.72001,196.72001,11.4803,247.112422,218.209695,145.956044,117.053317,...,0.0,0.0,201.921238,189.255071,1,1,1,126,0.826354,-0.563151


In [12]:
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models

WINDOW_SIZE = 30

def make_sequences(df, feature_cols, target_col):
    X, y = [], []
    for i in range(len(df) - WINDOW_SIZE):
        seq = df[feature_cols].iloc[i : i + WINDOW_SIZE].values
        X.append(seq)
        y.append(df[target_col].iloc[i + WINDOW_SIZE])
    return np.array(X), np.array(y)

feature_cols = ['Composite Weather Variable - Actual', 'Demand Actual, NTS, D+1', 'Demand Forecast, NTS, hourly update', 'Interconnector - Daily Flow', 'Medium Storage - Actual Stock',
              'Medium Storage - Stock Level at Max Flow', 'Predicted Closing Linepack (PCLP1)', 
              'SAP', 'SMPBuy',	'SMPSell', 
              'Storage - Daily Flow','Storage - Delivery', 'Storage, Medium Range, Stock Levels', 'System Entry Flows, National, Forecast', 'System Entry Flows, National, Physical',
              'Day of Week','Is Weekday','Next Day Is Weekday','Day of Year']


target_col  = 'SAP'
X, y = make_sequences(df, feature_cols, target_col)

print(X[250])
print(y[250])

train_size = int(0.7 * len(X))
val_size   = int(0.1 * len(X))

X_train, y_train = X[:train_size], y[:train_size]
X_val,   y_val   = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test,  y_test  = X[train_size+val_size:], y[train_size+val_size:]
print(f"Training records: {X_train.shape[0]}")
print(f"Validation records: {X_val.shape[0]}")
print(f"Test records: {X_test.shape[0]}")

#scaling

n_feats = X_train.shape[2]
scaler = StandardScaler()
X_train_2d = X_train.reshape(-1, n_feats)
scaler.fit(X_train_2d)

def scale_split(X):
    X_2d = X.reshape(-1, n_feats)
    Xs = scaler.transform(X_2d)
    return Xs.reshape(-1, WINDOW_SIZE, n_feats)

X_train = scale_split(X_train)
X_val   = scale_split(X_val)
X_test  = scale_split(X_test)



model = models.Sequential([
    # First LSTM layer returns full sequences so you can stack
    layers.LSTM(128, 
                return_sequences=True, 
                input_shape=(WINDOW_SIZE, n_feats)),
    layers.Dropout(0.2),

    # Second LSTM layer (doesn't return sequences)
    layers.LSTM(64),
    layers.Dropout(0.2),

    # A small dense “head” to predict next price
    layers.Dense(32, activation='relu'),
    layers.Dense(1)  # single real-valued output
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='mse',
    metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')]
)

model.summary()

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5
    )
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    callbacks=callbacks
)

# 5. Evaluate model RMSE on test set
eval_results = model.evaluate(X_test, y_test, batch_size=32, return_dict=True)
model_rmse = eval_results['rmse']

# 6. Compute naïve “tomorrow = today” RMSE
#    we know 'SAP' is at index:
target_idx = feature_cols.index(target_col)
#    take last timestep of SAP from each test sequence:
y_pred_naive = X_test[:, -1, target_idx]
naive_rmse = np.sqrt(np.mean((y_test - y_pred_naive)**2))

# 7. Print comparison
print(f"Model RMSE: {model_rmse:.4f}")
print(f"Naïve ‘tomorrow = today’ RMSE: {naive_rmse:.4f}")

[[4.17000000e+00 3.66755521e+02 3.66624661e+02 8.73802500e+01
  6.98598287e+01 1.20539677e+04 3.61734180e+02 2.13920000e+00
  2.17770000e+00 2.10070000e+00 3.59189300e+01 1.83655700e+01
  1.20539677e+04 3.64579380e+02 3.58993205e+02 5.00000000e+00
  0.00000000e+00 0.00000000e+00 9.00000000e+00]
 [4.14000000e+00 3.47255467e+02 3.48694237e+02 8.56777300e+01
  6.75815802e+01 1.16608672e+04 3.64646208e+02 2.08810000e+00
  2.12660000e+00 2.03020000e+00 2.14951600e+01 9.99965000e+00
  1.16608672e+04 3.50538210e+02 3.39637023e+02 6.00000000e+00
  0.00000000e+00 1.00000000e+00 1.00000000e+01]
 [4.09000000e+00 3.27737849e+02 3.27989177e+02 7.43592000e+01
  6.68779507e+01 1.15394594e+04 3.64945794e+02 2.22000000e+00
  2.25850000e+00 2.08140000e+00 1.57294800e+01 8.73999000e+00
  1.15394594e+04 3.29124762e+02 3.30097630e+02 0.00000000e+00
  1.00000000e+00 1.00000000e+00 1.10000000e+01]
 [4.02000000e+00 3.49945729e+02 3.49664137e+02 8.77308000e+01
  6.68842573e+01 1.15405476e+04 3.65249599e+02 2.6

c:\Users\mike\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 30, 128)        │        75,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 30, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 127,297 (497.25 KB)

 Trainable params: 127,297 (497.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 17s 85ms/step - loss: 14.5451 - rmse: 3.7498 - val_loss: 0.9086 - val_rmse: 0.9532 - learning_rate: 0.0010
Epoch 2/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - loss: 2.1525 - rmse: 1.4635 - val_loss: 0.4241 - val_rmse: 0.6512 - learning_rate: 0.0010
Epoch 3/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - loss: 1.4509 - rmse: 1.2001 - val_loss: 0.3594 - val_rmse: 0.5995 - learning_rate: 0.0010
Epoch 4/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - loss: 1.1765 - rmse: 1.0798 - val_loss: 0.2185 - val_rmse: 0.4675 - learning_rate: 0.0010
Epoch 5/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - loss: 0.7302 - rmse: 0.8527 - val_loss: 0.4708 - val_rmse: 0.6862 - learning_rate: 0.0010
Epoch 6/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - loss: 0.7415 - rmse: 0.8602 - val_loss: 0.2664 - val_rmse: 0.5162 - learning_rate: 0.0010
Epoch 7/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 55ms/step - loss: 0.7817 - rmse: 0.8800 - val_loss: 0.2329 - val_rmse: 0.4826 - learning_rat